# <font color="white"> 解析OJT用ツール(Jupyter Notebook)<font>

In [1]:
%%capture installer_log
!pip install xgboost
!apt-get update && apt-get install -y build-essential
!python -m pip install shap
!pip install pymc3
!pip install tqdm
!pip --no-cache-dir install ctgan
!pip install pandas-profiling
!pip install msoffcrypto-tool
!pip install japanize-matplotlib

In [2]:
import sagemaker
from tybmilib import prep
from tybmilib import vis
from tybmilib import datamgmt
from tybmilib import modeling
from tybmilib import paramsearch
from tybmilib.logmgmt import create_logger, stop_watch


## 初期パラメーター設定（★要変更）

---

In [3]:
# S3に格納された分析対象データのURL
s3uri_original_data = ['s3://mi-nu/210806更新_NU3高木_HC密着性rev.6_粒子_HCall,ink,AB,haze.xlsx']

# アウトプット格納バケット
# S3上に個人用ディレクトリを作成するため、お名前とテーマを変更して下さい。
experiment_ID = '0006'
create_logger(experiment_ID)
s3_bucket_path = 'sagemaker/takagi/HC'
role = sagemaker.get_execution_role()

# <font color="white"> 1. データ投入・確認<font>

#### <font color="white"> データ読み込み<font>

In [4]:
# データ読み込み
# 実験データのExcelファイルにパスワードが設定されている場合、データ読み込み時に入力が必要となります。
df_s3 = prep.read_s3_bucket_data(s3uri_original_data)

Please enter file password. ········


読み込みデータのカラム名：Name,nB,pH,nsp,AV,Tg,Mn,Mw,Acid_NDC,Acid_TPA,Acid_IPA,Acid_SA,Acid_AA,Acid_GCM,Gly_EG,Gly_HD,Gly_BD,Gly_DEG,Gly_NPG,Gly_PG,Gly_MPD,Gly_DMH,ink,HC,HC_48h,HC_240h,HC_boil,AB_standard,AB_press,AB_water,HAZE,TT


## パラメーター設定（★要変更）

In [5]:
# 本ステップで必要な設定項目
# 入力データに含まれる不要カラムの指定
drop_cols = ['Name','nB','pH','AV','Tg','Mn','Mw','HC_240h','HC_boil','AB_press','AB_water','TT']
# 目的変数
objectives = ['ink','HC','HC_48h','AB_standard','HAZE']

# データ可視化方法
# profiles = pandas-profiles / pairplot = N×N散布図行列 / pairplot_1by1 = 1×N散布図行列 / correlation_matrix = 相関変数行列
show_plot_method=['correlation_matrix']

## 処理実行

#### <font color="white"> 不要列削除 & 可視化<font>

In [6]:
# 不要列の削除
# 実験データ内で不要カラムがある場合には、colsのリスト内にカラム名を追加し削除して下さい。
df_reg = prep.drop_cols(df_s3,drop_cols)

# データ可視化
vis.show_plot(df_reg,objectives,show_plot_method)

  0%|          | 0/2 [00:00<?, ?it/s]

=========outputフォルダへの格納データ=========
['/root/output/correlation_matrix_all.png', '/root/output/correlation_matrix_only_objectives.png']


#### <font color="white"> 学習データ作成<font>

In [7]:
# 学習データの作成
# 目的変数分の学習データを作成し、S3のSageMaker用ディレクトリに格納
traindata_path_list = prep.create_multi_traindata(df_reg,objectives=objectives)
s3_uri_list = datamgmt.S3Dao().upload_data(traindata_path_list,s3_bucket_path=s3_bucket_path)

# 次のステップで使用する情報を提示
prep.present_info(s3_bucket_path,role,objectives,s3_uri_list,traindata_path_list,experiment_ID)

=========dataフォルダへの格納データ=========
目的変数：ink
学習データ：train_ink.csv
目的変数：HC
学習データ：train_HC.csv
目的変数：HC_48h
学習データ：train_HC_48h.csv
目的変数：AB_standard
学習データ：train_AB_standard.csv
目的変数：HAZE
学習データ：train_HAZE.csv
説明変数のみデータ：train(only_x).csv
目的変数のみデータ：train(only_y).csv
=========s3指定バケットへの格納データ=========
s3://sagemaker-ap-southeast-1-375869297825/sagemaker/takagi/HC/train_ink.csv
s3://sagemaker-ap-southeast-1-375869297825/sagemaker/takagi/HC/train_HC.csv
s3://sagemaker-ap-southeast-1-375869297825/sagemaker/takagi/HC/train_HC_48h.csv
s3://sagemaker-ap-southeast-1-375869297825/sagemaker/takagi/HC/train_AB_standard.csv
s3://sagemaker-ap-southeast-1-375869297825/sagemaker/takagi/HC/train_HAZE.csv
------------------------------
#=========【途中再起動した場合、別セルに貼り付け、実行】以下の情報は、次セクションでも利用します。=========
objectives = ['ink', 'HC', 'HC_48h', 'AB_standard', 'HAZE']
s3_uri_list = ['s3://sagemaker-ap-southeast-1-375869297825/sagemaker/takagi/HC/train_ink.csv', 's3://sagemaker-ap-southeast-1-375869297825/sagemaker/takagi/HC

---

# <font color="white"> 2. モデル構築・評価<font>

## パラメーター設定（★要変更）

In [8]:
# 本ステップで必要な設定項目
# 機械学習モデルの指定
#'Regression|BinaryClassification|MulticlassClassification'
#problemtype = ''

# モデル評価指標の設定
# Regressionの場合:'MSE' / BinaryClassificationの場合:'F1' / MulticlassClassificationの場合:'F1macro'を指定
#metrics = '' 

## 処理実行

#### <font color="white"> モデル学習<font>

In [9]:
#%%capture model_log
#%%time
#mlmodel = modeling.SagemakerCtrl(s3_bucket_path,role,experiment_ID,problemtype=problemtype,metrics=metrics)
#model_list,ep_list = mlmodel.fit_multi_model(objectives,s3_uri_list)

In [10]:
#model_log.show()

#### <font color="white"> モデル評価<font>

In [11]:
# Shap値出力
#mlmodel.analyze_multi_model(objectives,s3_uri_list,traindata_path_list,model_list)
# 線形モデルでの係数
#mlmodel.estimate_multi_coefficients(objectives,traindata_path_list)
# テストデータでの評価値
#mlmodel.estimate_multi_testdata(objectives,ep_list,traindata_path_list)

# 次のステップで使用する情報を提示
#mlmodel.present_info(objectives,ep_list,model_list,s3_bucket_path)

---

# <font color="white"> 3. パラメーター探索<font>

## パラメーター設定（★要変更）

#### <font color="white"> 途中実行する場合での引継ぎ情報<font>

In [12]:
#=========【途中再起動した場合、別セルに貼り付け、実行】以下の情報は、次セクションでも利用します。=========
objectives = ['HC_48h','AB_standard']
ep_list = ["ep-takagi-2-0006","ep-takagi-3-0006"]
model_list = ["ml-takagi-2-0006","ml-takagi-3-0006"]
s3_bucket_path = 'sagemaker/takagi/HC'
problemtype = 'Regression'

#### <font color="white"> 制約条件設定<font>

In [13]:
# 制約設定
# 変数を固定　GCM量は７で固定
fixed = [{'target':'Acid_GCM','value':7}]
# e.g. fixed = [{'target':'G2','value':30},{'target':'A3','value':30},{'target':'A4','value':30},{'target':'A6','value':120}]

# 変数の値範囲を指定 還元粘度は０～１
limit = [{'target':'nsp','range':(lambda x : 1>x>0)}]
# e.g. limit = [{'target':'G3','range':(lambda x : 50>x)},{'target':'G2','range':(lambda x : 10>x>7)},{'target':'A3','range':(lambda x : 20>x)}]

# 変数間の合計値を設定
total = [{'target':['Acid_NDC','Acid_TPA','Acid_IPA','Acid_SA','Acid_AA','Acid_GCM'],'total':100},{'target':['Gly_EG','Gly_HD','Gly_BD','Gly_DEG','Gly_NPG','Gly_PG','Gly_MPD','Gly_DMH'],'total':100}]
# e.g. total = [{'target':['A1','A2','A3','A4','A5','A6'],'total':100},{'target':['G1','G2','G3','G4','G5','G6'],'total':100}]

# 組成候補の選択方法を設定 Acidは１～３種類（GCMは必須）、Glyも１～３種類
combination = [{'target':['Acid_NDC','Acid_TPA','Acid_IPA','Acid_SA','Acid_AA','Acid_GCM'],'range':(lambda x : 0<x<4)},{'target':['Gly_EG','Gly_HD','Gly_BD','Gly_DEG','Gly_NPG','Gly_PG','Gly_MPD','Gly_DMH'],'range':(lambda x : 0<x<4)}]
# e.g. combination = [{'target':['A1','A2','A3','A4','A5','A6'],'range':(lambda x : x==3)},{'target':['G1','G2','G3','G4','G5','G6'],'range':(lambda x : 1<x<4)}]

# 組成時の材料比を設定
ratio = []
# e.g. ratio = [{'target':['A7','G3'],'ratio':(lambda x : x*1.5)},{'target':['G2','A4'],'ratio':(lambda x : x*2)}]

#### <font color="white"> 探索条件設定<font>

In [14]:
# 探索手法を設定
search_method = ['Simulate','Search_Cluster','Search_Pareto']

# 目的変数の目標値を設定
objectives_target = [{'target':'HC_48h','range':(lambda x : x>=4.5)},{'target':'AB_standard','range':(lambda x : x>=4)}]
# e.g. objectives_target = [{'target':'Tg','range':(lambda x : 7>x>3)},{'target':'Tm','range':(lambda x : 130>x>0)}]

## 処理実行

#### <font color="white"> 実験サンプル生成<font>

In [ ]:
# 制約条件の設定
sb = paramsearch.Search_Boundary(problemtype)
boundary_setting = sb.set_boundary(fixed=fixed,limit=limit,combination=combination,total=total,ratio=ratio)
# 設定条件の表示
sb.describe(boundary_setting)

# 設定条件に基づくサンプル生成
samples = sb.create_samples(boundary_setting,objectives,ep_list)

{'combination': [{'range': <function <lambda> at 0x7fe4c05fb170>, 'target': ['Acid_NDC', 'Acid_TPA', 'Acid_IPA', 'Acid_SA', 'Acid_AA', 'Acid_GCM']}, {'range': <function <lambda> at 0x7fe4c05fb200>, 'target': ['Gly_EG', 'Gly_HD', 'Gly_BD', 'Gly_DEG', 'Gly_NPG', 'Gly_PG', 'Gly_MPD', 'Gly_DMH']}],
 'fixed': [{'target': 'Acid_GCM', 'value': 7}],
 'limit': [{'range': <function <lambda> at 0x7fe4c0e1f680>, 'target': 'nsp'}],
 'total': [{'target': ['Acid_NDC', 'Acid_TPA', 'Acid_IPA', 'Acid_SA', 'Acid_AA', 'Acid_GCM'], 'total': 100}, {'target': ['Gly_EG', 'Gly_HD', 'Gly_BD', 'Gly_DEG', 'Gly_NPG', 'Gly_PG', 'Gly_MPD', 'Gly_DMH'], 'total': 100}]}


  0%|          | 0/4.0 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

# <font color="white"> 探索<font>

In [ ]:
ps = paramsearch.Search(ep_list,objectives,problemtype)
ps.search_samples(samples,objectives_target,search_method)

=========outputフォルダへの格納データ=========
=====【simulate】探索結果:条件を満たす実験サンプル：samples_Simulate.csv=====
=========outputフォルダへの格納データ=========
=====【Search_Cluster】探索結果:条件を満たす実験サンプル：samples_Search_Cluster.csv=====
クラスタリング結果(クラスター毎の平均値)：Clustering_by_Kmeans.csv
各特徴量を2次元に次元圧縮した場合でのクラスタリング状況の描画：Clustering_by_Kmeans.png
=========outputフォルダへの格納データ=========
パレート解となるサンプル群：pareto_optimal_samples.csv
パレート解の描画(使用2変数ごとに2次元で描画)：pareto_optimal_samples_使用変数.png


# <font color="white"> 4. 作成物一括削除<font>

## パラメーター設定（★要変更）

In [ ]:
# 途中実行する場合、モデル評価での出力データをコピーし実行
ep_list = ['']
model_list = ['']
s3_bucket_path = ''

## 処理実行

In [ ]:
prep.delete_resources(s3_bucket_path,model_list,ep_list)

ParamValidationError: Parameter validation failed:
Invalid length for parameter Key, value: 0, valid min length: 1